In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
###
!pip install torch
###
!pip install torchvision
!pip install d2l==1.0.0b0
!conda install -c conda-forge torchmetrics

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:26
🔁 Restarting kernel...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 105.2 MB/s eta 0:00:00
     ━━━━━━

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ 

In [ ]:
!pip install wandb -qU
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: amoseley018. Use `wandb login --relogin` to force relogin


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import torch
import torchvision
from torchvision import transforms
from torch import nn, Tensor
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchsummary import summary
from d2l import torch as d2l
from pathlib import Path
import sys
import nibabel as nib
import numpy as np
import gc
import skimage
import h5py
import math
import PIL
from PIL import Image

/usr/local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, weight0=1, weight1=1, gamma=0):
        super().__init__()

        self.weight0 = weight0
        self.weight1 = weight1
        self.gamma = gamma

    def forward(self, input, target):
        loss = 0
        for i, el in enumerate(input):
            if target[i] == 1:
                loss += torch.log(el) * (abs(1 - el) ** self.gamma) * self.weight1
            else:
                loss += torch.log(1 - el) * (abs(0 - el) ** self.gamma) * self.weight0

        return -1 * loss / len(input)

In [ ]:
class BalancedCELoss(nn.Module):
    def __init__(self, weight0=1, weight1=1):
        super().__init__()

        self.weight0 = weight0
        self.weight1 = weight1

    def forward(self, input, target):
        loss = 0
        for i, el in enumerate(input):
            if target[i] == 1:
                loss += torch.log(el) * self.weight1
            else:
                loss += torch.log(1 - el) * self.weight0

        return -1 * loss / len(input)

In [ ]:
class LITSBinaryDataset(Dataset):
    def __init__(self, fileName):
        super().__init__()

        self.file = h5py.File(fileName, 'r')

        self.length = len(list(self.file.keys()))

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        data = self.file["Slice" + str(idx)]["Slice"]
        label = self.file["Slice" + str(idx)].attrs.get("ImageLabel")

        result = []

        result.append(torch.Tensor(data[...]).unsqueeze(0))
        result.append(torch.Tensor(label).squeeze(0))

        return result

    def closeFile(self):
        self.file.close()

In [ ]:
class convBlock(nn.Module):
    def __init__(self, inChannels, outChannels, batchNorm, strides, layerMean, layerDev) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(inChannels, outChannels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(outChannels, outChannels, kernel_size=3, padding=1)

        nn.init.normal_(self.conv1.weight, mean=layerMean, std=layerDev)
        nn.init.normal_(self.conv2.weight, mean=layerMean, std=layerDev)

        if(batchNorm):
            self.bn1 = nn.BatchNorm2d(outChannels)
        else:
            self.bn1 = False

    def forward(self, X):
        Y = self.conv1(X)

        if(self.bn1):
            Y = self.bn1(Y)

        Y = F.relu(Y)

        return torch.Tensor(F.relu(self.conv2(Y)))

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, inChannels, outChannels, strides) -> None:
        super().__init__()

        self.conv = convBlock(inChannels, outChannels, True, strides, 0, 0.025)
        self.pool = nn.MaxPool2d(2, stride=strides)

    def forward(self, X):

        Y = self.conv.forward(X)

        #return torch.Tensor(self.pool(Y)), Y
        return self.pool(Y)

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, inChannels, outChannels, strides) -> None:
        super().__init__()

        self.convTrans = nn.ConvTranspose2d(inChannels, outChannels, 2, stride=strides, padding=1)
        self.conv = convBlock(outChannels, outChannels, True, strides, 0, 0.25)

    def forward(self, X, skipFeatures):
        Y = self.convTrans(X)
        Y = torch.cat(X, skipFeatures)
        return self.conv(Y)

In [ ]:
def evaluate_accuracy(net, testIter, lossFunc, device=None):
    net.eval()

    metric = d2l.Accumulator(3)

    with torch.no_grad():
        for X, y in testIter:
            X = X.to(device)
            y = y.to(device)

            yhat = net(X)

            prediction = torch.round(yhat).to(device)

            metric.add(d2l.accuracy(prediction, y) / torch.numel(y), torch.numel(y), lossFunc(yhat, y))

    return metric[0] / metric[1], metric[2] / len(testIter)

In [ ]:
def train(net: nn.Sequential, trainIter, testIter, numEpochs, startEpoch, learnRate, batchSize, device: torch.device, startDim, epochsToDouble, modelFileName, epochsToSave, progressive=False, lossFunc = nn.BCEWithLogitsLoss()):
    print(f"Training on {device}")
    
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=learnRate)
    numBatches = len(trainIter)

    imageFunc = transforms.ToPILImage()
    tensorFunc = transforms.ToTensor()

    bestValLoss = float('inf')

    currDim = startDim
    for epoch in range(startEpoch, numEpochs):
        if (epoch + 1) % epochsToDouble == 0 and progressive:
            currDim *= 2
            learnRate /= 4
            optimizer = torch.optim.SGD(net.parameters(), lr=learnRate)
        
        net.train()
        metric = d2l.Accumulator(2)

        for i, (X, y) in enumerate(trainIter):
            optimizer.zero_grad()
            y = y.to(device)

            if progressive:
                realX = []

                for slice in X:
                    slice = imageFunc(slice.squeeze(0)).resize((currDim, currDim), PIL.Image.BILINEAR)
                    slice = tensorFunc(slice).to(device)
                    realX.append(slice)

                realX = torch.stack(realX).to(device)
            else:
                realX = X.to(device)

            yhat = net(realX).squeeze(1)

            prediction = torch.round(yhat).to(device)

            l = lossFunc(yhat, y)

            #print(f"Loss: {l.item()} Predictions: {yhat.tolist()} Labels: {y.tolist()}")

            l.backward()
            optimizer.step()

            metric.add(l, d2l.accuracy(prediction, y) / batchSize)

        if (epoch + 1) % epochsToSave == 0:
            torch.save(net.state_dict(), modelFileName + "Epoch" + str(epoch))

        validationAcc, validationLoss = evaluate_accuracy(net, testIter, device)

        if validationLoss < bestValLoss:
            bestValLoss = validationLoss
            torch.save(net.state_dict(), modelFileName + "Best")

        print(f"Train Acc: {metric[1] / numBatches} Validation Acc: {validationAcc} Loss: {metric[0] / numBatches}")

        wandb.log({"Train Acc": metric[1] / numBatches,
                   "Validation Acc": validationAcc,
                   "Loss": (metric[0] / numBatches)
                  })

In [ ]:
gc.collect()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cpu")

startEpoch = 0
networkFileName = ""
fileSaveName = "/content/drive/MyDrive/LiverClassifier"

batchSize = 6
learnRate = 0.0003
epochs = 100
startDim = 32
epochsToDouble = 25
epochsToSave = 10

#lossFunc = BalancedCELoss(weight0=1, weight1=1.5)
lossFunc = FocalLoss(weight0=1, weight1=2, gamma=2)
#lossFunc = nn.BCEWithLogitsLoss()

wandb.init(project="LiverClassifier",
           name="",
           config={
               "BatchSize":batchSize,
               "LearnRate":learnRate,
               "Epochs":epochs,
               "StartDimension":startDim,
               "EpochsToDouble":epochsToDouble
           })


block1 = EncoderBlock(1, 64, 1)
block2 = EncoderBlock(64, 128, 1)
block3 = EncoderBlock(128, 256, 1)
block4 = EncoderBlock(256, 512, 1)
block5 = EncoderBlock(512, 1024, 1)

net = nn.Sequential(block1, block2, block3, nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(), nn.Linear(256, 1), nn.Sigmoid())
#print(summary(net, (1, 512, 512)))

if networkFileName != "":
    net.load_state_dict(torch.load(networkFileName))

print("Intialized model")

trainDataset = LITSBinaryDataset("drive/MyDrive/College/Spring2023/MachineLearningResearch/Datasets/TrainDataset.hdf5")
validationDataset = LITSBinaryDataset("drive/MyDrive/College/Spring2023/MachineLearningResearch/Datasets/ValidationDataset.hdf5")
testDataset = LITSBinaryDataset("drive/MyDrive/College/Spring2023/MachineLearningResearch/Datasets/TestDataset.hdf5")

print("Dataset loaded")

trainIter = DataLoader(trainDataset, batch_size=batchSize, shuffle=True)
validationIter = DataLoader(validationDataset, batch_size=batchSize)
testIter = DataLoader(testDataset, batch_size=batchSize)

train(net, trainIter, validationIter, epochs, startEpoch, learnRate, batchSize, device, startDim, epochsToDouble, fileSaveName, epochsToSave, progressive=True, lossFunc=lossFunc)

testAcc = evaluate_accuracy(net, testIter, device=device)
print(f"Test Accuracy: {testAcc}")

trainDataset.closeFile()
validationDataset.closeFile()
testDataset.closeFile()

wandb.finish()

Intialized model
Dataset loaded
Training on cuda
Train Acc: 0.4790167865707437 Test Acc: 0.24496644295302014 Loss: 0.24188000783860256
Train Acc: 0.48141486810551554 Test Acc: 0.24496644295302014 Loss: 0.238006404085125
Train Acc: 0.4814148681055157 Test Acc: 0.24496644295302014 Loss: 0.2356281654654647
Train Acc: 0.4814148681055154 Test Acc: 0.24496644295302014 Loss: 0.2346559464395475
Train Acc: 0.4814148681055157 Test Acc: 0.24496644295302014 Loss: 0.23256783117707686
Train Acc: 0.4814148681055152 Test Acc: 0.24496644295302014 Loss: 0.23079862359830802
Train Acc: 0.4814148681055157 Test Acc: 0.24496644295302014 Loss: 0.22869565445098947
Train Acc: 0.48141486810551554 Test Acc: 0.24496644295302014 Loss: 0.2267857229323696
Train Acc: 0.4814148681055157 Test Acc: 0.24496644295302014 Loss: 0.22452764272046605
Train Acc: 0.4820143884892083 Test Acc: 0.24496644295302014 Loss: 0.22121281380490435
Train Acc: 0.48381294964028787 Test Acc: 0.24496644295302014 Loss: 0.2181652871717652
Train Ac

Loss,███▇▇▇▆▅▅▄▅▄▄▄▃▃▃▃▂▂▃▃▂▂▂▂▂▂▂▁▂▂▂▂▂▁▁▁▁▁
Test Acc,▁▁▁▁▁▇▁▇▁▁▁▁▇▁▇▇▇▁▇▁▆▁▇▇▇▆▇▇▅▁▇▇▁▇██▁▇▇▇
Train Acc,▁▁▁▁▁▁▃▄▆▆▅▆▆▆▇▇▇▇▇█▇▇▇▇▇█████▇▇████████
Loss,0.01334
Test Acc,0.75503
Train Acc,0.90947
